<a href="https://colab.research.google.com/github/raduga256/tensorflow-dp-learning-dev/blob/main/v2.7/code_example_schuler_baseline_various_mixes_v2_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
monitor='val_accuracy'
epochs=30
batch_size=16
input_shape=(128, 128, 3) # please resize it to (224,224,3) if you have enough RAM
Verbose=True

This source code requires a **HIGH RAM** machine.

You might need to install this on your system:

apt-get install python3-opencv git

In [2]:
import os

if not os.path.isdir('k'):
  !git clone -b development14 https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1751, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 1751 (delta 106), reused 98 (delta 48), pack-reused 1588
Receiving objects: 100% (1751/1751), 15.62 MiB | 23.41 MiB/s, done.
Resolving deltas: 100% (1219/1219), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/k
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cai: filename=cai-0.1.5-py3-none-any.whl size=57415 sha256=51ec0be3e07008cf7635f6f5d13bfc3c705b6eb58cb4534c4616fed38b92298

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

import skimage
print('skimage version',  skimage.__version__)
import tensorflow as tf
print(tf.__version__)
import sys
import cai
import cai.datasets
import cai.densenet
import cai.util
import cai.models

In [ ]:
url_zip_file="https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file, 
    expected_folder_name=expected_folder_name, Verbose=Verbose)

In [ ]:
import random
import os
import multiprocessing
import glob
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
import sklearn.utils.class_weight
from tensorflow import keras
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import utils as np_utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!rm -r plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves -R
data_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"
print(os.listdir(data_dir))

In [ ]:
def compiled_two_path_inception_v3(
  classes=1000, 
  input_shape=input_shape,
  two_paths_first_block=False,
  two_paths_second_block=False,
  max_mix_idx=10):
  base_model = cai.models.two_path_inception_v3(
    include_top=False,
    weights=None,
    input_shape=input_shape,
    pooling=None,
    two_paths_first_block=two_paths_first_block,
    two_paths_second_block=two_paths_second_block,
    max_mix_idx=max_mix_idx)
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(38, name='preprediction')(x)
  predictions = Activation('softmax',name='prediction')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  model.compile(loss='categorical_crossentropy',
  optimizer = 'sgd',
  metrics=['accuracy'])
  return model

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7, root_dir=data_dir, lab=False, 
  verbose=Verbose, bipolar=False, base_model_name='plant_leaf',
  training_size=0.6, validation_size=0.2, test_size=0.2,
  target_size=(input_shape[0],input_shape[1]), 
  has_training=True, has_validation=True, has_testing=True, 
  smart_resize=True)

In [ ]:
print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

In [ ]:
for max_mix_idx in [5]: # range(-1,10,1):
    basefilename = 'Schuler-baseline-v2.7-'+str(max_mix_idx)
    print('Running: '+basefilename)
    model = compiled_two_path_inception_v3(
      classes=38,
      input_shape=input_shape,
      two_paths_first_block=False,
      two_paths_second_block=False,
      max_mix_idx=max_mix_idx)
    best_result_file_name = basefilename+'-best-result.hdf5'
    save_best = tensorflow.keras.callbacks.ModelCheckpoint(
      filepath=best_result_file_name, 
      monitor=monitor, 
      verbose=1, 
      save_best_only=True, 
      save_weights_only=False, 
      mode='max', 
      save_freq='epoch')
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,
      validation_data=(val_x,val_y),
      callbacks=[save_best], 
      class_weight = classweight
    )
    print('Testing Last Model: '+basefilename)
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Best Model Results: '+basefilename)
    model = keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Finished: '+basefilename)